<font color='Sucees'> 
### <u>LEGEND</u>
<br />
<div class="alert alert-block alert-success">
<b>Acceptable:</b> Looks good to me (LGTM).
</div>
<div class="alert alert-block alert-danger">
<b>Needs rework:</b> The block needs to be reworked.
</div>
</font>

# SQL Project

Due to the Corona Virus, people have more time on their hands and thus started using apps to read books. As a junior data analyst, database of one of the services competing in that market has been provided for analysis. 

The dataset contains data on books, publishers, authors and customer ratings and reviews of the books. After analyzing this data, it will be used to generate a value proposition of a new product. 

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Print rows of each table

In [2]:
#Books Table
pd.io.sql.read_sql("SELECT * FROM books", con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [3]:
#Authors Table
pd.io.sql.read_sql("SELECT * FROM authors", con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [4]:
#Publishers Table
pd.io.sql.read_sql("SELECT * FROM publishers", con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [5]:
#Ratings Table
pd.io.sql.read_sql("SELECT * FROM ratings", con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [6]:
#Reviews Table
pd.io.sql.read_sql("SELECT * FROM reviews", con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


<div class="alert alert-block alert-success">
OK, agreed.
</div>

### Find the number of books released after January 1, 2000.

In [7]:
pd.io.sql.read_sql("""
                    SELECT COUNT(book_id) as number_of_books_released 
                    FROM books 
                    WHERE publication_date > '2000-1-1'
                    """, 
                   con = engine)

,number_of_books_released
0,819


There has been 819 books released after January 1, 2000.

### Find the number of user reviews and the average rating for each book.

In [8]:
pd.io.sql.read_sql("""
                    SELECT title,
                        subq.number_of_user_reviews,
                        subq.avg_rating
                    FROM
                        (SELECT COUNT(review_id) as number_of_user_reviews,
                            AVG(rating) as avg_rating,
                            reviews.book_id as book_id
                        FROM reviews
                        INNER JOIN ratings on reviews.book_id = ratings.book_id
                        GROUP BY reviews.book_id) as subq
                    INNER JOIN books on subq.book_id = books.book_id
                    ORDER BY subq.number_of_user_reviews DESC
                    """, 
                   con = engine)

,title,number_of_user_reviews,avg_rating
0,Twilight (Twilight #1),1120,3.662500
1,The Hobbit or There and Back Again,528,4.125000
2,The Catcher in the Rye,516,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,480,4.287500
...,...,...,...
989,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2,4.500000
990,The Burden of Proof (Kindle County Legal Thril...,2,3.500000
991,Executive Orders (Jack Ryan #8),2,3.500000
992,Ten Apples Up On Top!,2,4.000000


Reviews per book vary from just 2 reviews to 1120 reviews. Average ratings for each book also varies a lot.

<div class="alert alert-block alert-success">
    
<b>V2:</b> Agreed 👍👍👍
</div>

<div class="alert alert-block alert-danger">
<del>    
<b>V1:</b> Look 👇, such type of grouping brings a lot of review and rating ids into a book. Group `reviews` and `ratings` tables and calculate <b>count</b> and <b>average</b> by `book_id` before joining with `books`.
</div>

In [9]:
pd.io.sql.read_sql("""
                    SELECT title as book_title,
                            review_id, 
                            rating_id
                    FROM ratings
                    INNER JOIN books ON books.book_id = ratings.book_id 
                    INNER JOIN reviews on reviews.book_id = books.book_id
                    """, 
                   con = engine)

,book_title,review_id,rating_id
0,'Salem's Lot,2,1
1,'Salem's Lot,1,1
2,'Salem's Lot,2,2
3,'Salem's Lot,1,2
4,'Salem's Lot,2,3
...,...,...,...
26162,Zen and the Art of Motorcycle Maintenance: An ...,2790,6455
26163,Zen and the Art of Motorcycle Maintenance: An ...,2793,6456
26164,Zen and the Art of Motorcycle Maintenance: An ...,2792,6456
26165,Zen and the Art of Motorcycle Maintenance: An ...,2791,6456


<div class="alert alert-block alert-danger">
<del>    
Please check and rebuild the query. We have to count number of reviews but don't. I'd suggest to group `reviews` and `ratings` before joining with `books`.
</div>

### Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [10]:
pd.io.sql.read_sql("""
                    SELECT COUNT(title), publisher
                    FROM books 
                    INNER JOIN publishers on books.publisher_id = publishers.publisher_id 
                    WHERE num_pages > 50
                    GROUP BY publisher
                    ORDER BY count DESC

                    """, 
                   con = engine)

,count,publisher
0,42,Penguin Books
1,31,Vintage
2,25,Grand Central Publishing
3,24,Penguin Classics
4,19,Ballantine Books
...,...,...
329,1,Turtleback
330,1,Atheneum Books for Young Readers: Richard Jack...
331,1,Penguin Signet
332,1,Victor Gollancz


The publisher that has released the most books with more than 50 pages is Penguin Books.

<div class="alert alert-block alert-success">
    
LGTM
</div>

### Identify the author with the highest average book rating: look only at books with at least 50 ratings.

In [11]:
pd.io.sql.read_sql("""
                   
                    SELECT authors.author AS author,
                            AVG(ratings.rating) AS avg_rating, 
                            COUNT(rating) as number_of_ratings                                              
                    FROM books 
                    INNER JOIN authors on authors.author_id = books.author_id 
                    INNER JOIN ratings on ratings.book_id = books.book_id
                    GROUP BY authors.author
                    HAVING COUNT(rating) > 50
                    ORDER BY avg_rating DESC
                    """, 
                   con = engine)

,author,avg_rating,number_of_ratings
0,J.K. Rowling/Mary GrandPré,4.288462,312
1,Agatha Christie,4.283019,53
2,Markus Zusak/Cao Xuân Việt Khương,4.264151,53
3,J.R.R. Tolkien,4.240964,166
4,Roald Dahl/Quentin Blake,4.209677,62
5,Louisa May Alcott,4.203704,54
6,Rick Riordan,4.130952,84
7,Arthur Golden,4.107143,56
8,Stephen King,4.009434,106
9,John Grisham,3.971429,70


J.K. Rowling/Mary GrandPré is the author with the highest average rating.

<div class="alert alert-block alert-success">
    
<b>V2:</b> Well done 👍👍👍
</div>

<div class="alert alert-block alert-danger">
<del>    
<b>V1:</b> Your query is well enough but not completed. Use it as a subquery, group it by `author` and calculate `AVG(rating)`.
</div>

<div class="alert alert-block alert-danger">
<del>    
You listed books and authors by rating but didn't count the averge rating. Please have it done.
</div>

### Find the average number of text reviews among users who rated more than 50 books.

In [12]:
pd.io.sql.read_sql("""
                    SELECT AVG(subq.text) as avg_text_reviews
                    FROM
                    (SELECT username,
                            COUNT(text) as text
                    FROM reviews
                    WHERE username in
                    (SELECT username
                    FROM ratings 
                    GROUP BY username
                    HAVING count(rating_id) > 50) 
                    GROUP BY username) as subq
                    """, 
                   con = engine)

,avg_text_reviews
0,24.333333


<div class="alert alert-block alert-success">
    
<b>V2:</b> Perfect 👏👏👏
</div>

<div class="alert alert-block alert-danger">
<del>    
<b>V1:</b> Use this query 👇 as a subquery to select text reviews as `WHERE author IN subquery`.
</div>

In [13]:
pd.io.sql.read_sql("""
                    select username
    from ratings 
    group by username
    having count(rating_id) > 50 
                    """, 
                   con = engine)

,username
0,sfitzgerald
1,jennifermiller
2,xdavis
3,paul88
4,martinadam
5,richard89


<div class="alert alert-block alert-danger">
<del>    
The number is too high. The query is not correct. You should make it in 3 steps:
1. Select users who made more than 50 rates
2. Select reviews for these users and
3. And count average number of reviews per a user
</div>

The average number of text reviews among users who have rated more than 50 books is about 24 reviews.

# General Conclusion

After analyzing and evaluating the data, several conclusions can be made in order to help create a proposition. 

I would suggest to focus on having the publisher Penguin Books to be available since many of their works are being read. Also, JK Rowling's books are popular so they should also be ready for the users. It also appears that books published after January 1, 2000 are popular since they make up 819 of the 1000 books, so focusing on recent releases would be good. I think finding a way to increase the reviews for each book would help other users to choose books as well.

<div class="alert alert-block alert-success">
    
<b>V2:</b> Well performed project 👏👏👏
</div>